In [1]:
from osgeo import gdal, osr
from osgeo import osr
import geopandas as gpd
from osgeo import ogr
from shapely.geometry import Polygon, Point
from geopandas.geoseries import *
import re

In [2]:
import tarfile
import pandas as pd
import datetime as dt
import os
import glob
import subprocess
import os.path
from os import path
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from terrautils.spatial import scanalyzer_to_latlon

In [3]:
# Grabs the thermal raw data tar file for the Gantry's metadata for the images taken at the specified date
class JSON:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/raw_tars/{season}/flirIrCamera/flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
        command = f'tar -xvf flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)

# Finds the individual json files and adds them to the filepath (end up with a list of paths for each individual json file)
    def pathlist(season, date):
        json_data = JSON.get_tar(season, date)
        pathlist = Path(f"./flirIrCamera/{date}/").glob('**/*.json')
        JSON_path_list = []
        for path in pathlist:
            path_str = str(path)
            JSON_path_list.append(path_str)
        return JSON_path_list

# Uses the pathlists and searches each one
# Gathers the time, image name, gantry_x and gantry_y information from the metadata of each individual image and adds to dictionary
    def time_dict(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]'])
                gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]'])
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename, gantry_x, gantry_y] = "Date, Time, Gantry_x, Gantry_y"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)

# Searches through the dictionary created and creates a dataframe of the information in the dictionary
    def time_df(season, date):
        JSON_time_d = JSON.time_dict(season, date)
        JSON_time_df = pd.DataFrame.from_dict(JSON_time_d)
        JSON_time_df.columns = ['Date and Time', 'Image Name', 'Gantry_x', 'Gantry_y']
        return JSON_time_df

# Converts the gantry coordinates into GPS coordinates (using 'terrautils')
    def GPS_coord (season, date):
        data = JSON.time_df(season, date)
        gantry_x_pos = data['Gantry_x']
        gantry_y_pos = data['Gantry_y']
        GPS_latlon = scanalyzer_to_latlon(gantry_x_pos, gantry_y_pos)
        GPS_df = pd.DataFrame(GPS_latlon)
        GPS_latlon_df = GPS_df.transpose()
        GPS_latlon_df.columns = ['GPS_lat', 'GPS_lon']
        data['GPS_lat'] = GPS_latlon_df['GPS_lat']
        data['GPS_lon'] = GPS_latlon_df['GPS_lon']
        return data

In [4]:
image_gps = JSON.GPS_coord('season_10_yr_2020', '2020-03-03')

In [5]:
lat,lon = image_gps.loc[1]['GPS_lat'], image_gps.loc[1]['GPS_lon']

In [6]:
shp = gpd.read_file('/Users/sebastiancalleja/Desktop/season10_multi_latlon_geno.geojson')

In [7]:
point_list = []
poly_list = []

class Get_plot:
    def to_point (point_data, geom_data):
        for i, row in point_data.iterrows():
            lat = point_data['GPS_lat'].loc[i]
            lon = point_data['GPS_lon'].loc[i]
            points = Point(lon,lat)
            point_list.append(points)
            point_list_df = gpd.GeoDataFrame(point_list)
        return point_list_df

In [8]:
image_gps['Points'] = Get_plot.to_point(image_gps, shp)

In [9]:
#GPS_points.set_index(0)
#GPS_points.rename(columns={0: 'Points'}, inplace=True)

In [10]:
#data_all = gpd.GeoDataFrame(shp['geometry'])

In [11]:
#GPS_points.at[i,"plot"]

In [12]:
def intersection(point):
    within = point.within
    plot = None
    intersection_list = []
    for i, row in shp.iterrows():
        polygon = row['geometry']
        intersection = within(polygon)
        if intersection == True:
            plot = [row['ID']]
            intersection_list.append(plot)
    return intersection_list

In [13]:
image_gps["plot"] = None
for i, row in image_gps.iterrows():
    point = row['Points']
    plot = intersection(point)
    image_gps.at[i,'plot'] = plot

In [14]:
# Find points that had no intersecrtion with plots
# We want to create a polygon out of the image and see which plots they intersect (rather than use the center point)
no_intersection = image_gps.loc[(image_gps['plot'].str.len() == 0),:]

In [15]:
# finds the numberof plots each image got associated with
plot_total_list = []
for i, row in image_gps.iterrows():
    total_plots = len(image_gps['plot'].loc[i])
    plot_total_list.append(total_plots)

In [17]:
# adds the total plot number to the main df
total_plot_df = pd.DataFrame(plot_total_list)
total_plot_df.rename(columns={0: 'total_plots'}, inplace=True)
image_gps['total_plots'] = total_plot_df

In [18]:
image_gps['total_plots'].value_counts()

1    7285
0    1718
2     267
Name: total_plots, dtype: int64